# Fourier Neural Operator

In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import os

torch.manual_seed(2002)
np.random.seed(2002)

## Постановка задачи

В качестве входных данных сеть получает начальное условие для уравнения Бюргерса $$ {\frac {\partial u}{\partial t}}+u{\frac {\partial u}{\partial x}}=\nu {\frac {\partial ^{2}u}{\partial x^{2}}}$$ на интервале $[0, 0.5]$ и экстраполирует его на интервал $[0.5, 1]$. Размер равномерной сетки - 256 элементов.

## Этап 1: обучение FNO из пакета neuralop

На этом этапе задача будет решаться с официальной реализацией FNO из пакета neuralop.

### Загрузка и обработка данных

Путь к данным. Лучше поменять, так как я прописывал его относительно базовой директории окружения.

In [2]:
datasets_path = "./Task2_with_datasets/datasets"

In [3]:
from dataloaders.burgers import load_burgers_1dtime
from dataloaders.tensor_dataset import TensorDataset

При загрузке данным надо вытащить сами массивы данных, чтобы разбить их на x и y.

In [4]:
_, _, _, train, test = load_burgers_1dtime(
    data_path=os.path.join(datasets_path, "burgers"),
    n_train=400, n_test=100, batch_size=16, batch_size_test=16,
    temporal_length=1, spatial_length=256)

In [5]:
print(train.shape, test.shape)

torch.Size([400, 3, 1, 256]) torch.Size([100, 3, 1, 256])


In [6]:
train = torch.squeeze(train, dim=2)
test = torch.squeeze(test, dim=2)
print(train.shape, test.shape)

torch.Size([400, 3, 256]) torch.Size([100, 3, 256])


In [7]:
x_train = train[:, :, :128]
y_train =  torch.unsqueeze(train[:, 0, 128:], dim=1)

x_test = test[:, :, :128]
y_test = torch.unsqueeze(test[:, 0, 128:], dim=1)
train_db = TensorDataset(x_train, y_train)
train_loader = torch.utils.data.DataLoader(train_db, batch_size=32, shuffle=False)

test_db = TensorDataset(x_test, y_test)
test_loader = torch.utils.data.DataLoader(test_db, batch_size=16, shuffle=False)

### Создание модели

In [8]:
from neuralop.models import FNO
from neuralop.utils import count_params
from neuralop import LpLoss, H1Loss
import sys

Создадим 1-d FNO:

In [39]:
device = "cuda"

model = FNO(n_modes=(32,), in_channels=3, out_channels=1, n_layers=6, hidden_channels=32, projection_channels=64)
model = model.to(device)

n_params = count_params(model)
print(f'\nOur model has {n_params} parameters.')
sys.stdout.flush()

optimizer = torch.optim.Adam(model.parameters(),
                             lr=1e-3)

scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10000, gamma=1)
# scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=100, eta_min=5e-4)

h1loss = H1Loss(d=1, reductions='mean')
l2loss =LpLoss(d=1, p=2, reductions='mean')

train_loss = h1loss
eval_losses={'h1': h1loss, 'l2': l2loss}

print('\n### MODEL ###\n', model)
print('\n### OPTIMIZER ###\n', optimizer)
print('\n### SCHEDULER ###\n', scheduler)
print('\n### LOSSES ###')
print(f'\n * Train: {train_loss}')
print(f'\n * Test: {eval_losses}')
sys.stdout.flush()


Our model has 205249 parameters.

### MODEL ###
 FNO(
  (fno_blocks): FNOBlocks(
    (convs): FactorizedSpectralConv(
      (weight): ModuleList(
        (0-5): 6 x ComplexDenseTensor(shape=torch.Size([32, 32, 16]), rank=None)
      )
    )
    (fno_skips): ModuleList(
      (0-5): 6 x Conv1d(32, 32, kernel_size=(1,), stride=(1,), bias=False)
    )
  )
  (lifting): Lifting(
    (fc): Conv1d(3, 32, kernel_size=(1,), stride=(1,))
  )
  (projection): Projection(
    (fc1): Conv1d(32, 64, kernel_size=(1,), stride=(1,))
    (fc2): Conv1d(64, 1, kernel_size=(1,), stride=(1,))
  )
)

### OPTIMIZER ###
 Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    initial_lr: 0.001
    lr: 0.001
    maximize: False
    weight_decay: 0
)

### SCHEDULER ###

### LOSSES ###

 * Train: <neuralop.training.losses.H1Loss object at 0x7f54a0086bb0>

 * Test: {'h1': <neuralop.training.losses.H1Los

Создаем Trainer с коллбэком для Tensorboard и обучаем модель:

In [10]:
from callbacks.tensorboard_callback import TensorBoardCallback
from callbacks.trainer import Trainer

2023-12-02 14:22:07.327745: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-02 14:22:07.759744: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [54]:
exp_num = 6

In [50]:
tb_callback = TensorBoardCallback(log_dir=f'./logs/run{exp_num}')
trainer = Trainer(model=model, n_epochs=202,
                  device=device,
                  wandb_log=False,
                  log_test_interval=3,
                  use_distributed=False,
                  verbose=True,
                  callbacks=[tb_callback])

trainer.train(train_loader=train_loader, model=model, 
              output_encoder=None,
              test_loaders=test_loader,
              optimizer=optimizer,
              scheduler=scheduler,
              regularizer=False,
              training_loss=train_loss,
              eval_losses=eval_losses)

using standard method to load data to device.
using standard method to compute loss.
Training on regular inputs (no multi-grid patching).
Training on 400 samples
Testing on [100] samples         on resolutions ['test'].
Training on raw inputs of size x.shape=torch.Size([32, 3, 128]), y.shape=torch.Size([32, 1, 128])
.. patched inputs of size x.shape=torch.Size([32, 3, 128]), y.shape=torch.Size([32, 1, 128])
Raw outputs of size out.shape=torch.Size([32, 1, 128])
.. Processed (unpatched) outputs of size out.shape=torch.Size([32, 1, 128])
[0] time=0.16, avg_loss=2.0446, train_err=1.0325, test_h1=1.0377, test_l2=1.0858
[3] time=0.12, avg_loss=1.9240, train_err=0.9716, test_h1=0.9322, test_l2=0.9257
[6] time=0.12, avg_loss=1.6995, train_err=0.8582, test_h1=0.7847, test_l2=0.8255
[9] time=0.12, avg_loss=1.5815, train_err=0.7986, test_h1=0.7334, test_l2=0.8382
[12] time=0.12, avg_loss=1.5290, train_err=0.7722, test_h1=0.6954, test_l2=0.7797
[15] time=0.12, avg_loss=1.3320, train_err=0.6726, t

Посмотрим на результаты экспериментов в Tensorboard.

In [45]:
!tensorboard --logdir=logs

2023-12-02 13:33:06.382849: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-02 13:33:06.802559: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-12-02 13:33:07.359897: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-02 13:33:07.360608: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA n

Графики для одного из запусков. Обратите внимание, функция потерь на обучении и тесте усреднена (reductions='mean'). 

![](imgs/1_train.png)

![](imgs/1_test.png)

**Выводы:**

1. FNO достаточно хорошо справился с задачей, MSE на тесте составила около $4 \cdot 10^{-2}$, что неплохо для данной задачи \
(т.к. нет никакой дополнительной физ.информации про начальные условия);

2. Кривая потерь на обучении и тесте достаточно гладкая, что говорит о стабильном обучении FNO с подобранными гиперпараметрами.

## Этап 2: написание собственной реализации FNO

На данном этапе требуется реализовать FNO собственноручно и провести исследование о влиянии количества гармоник на качество обучения.

In [ ]:
from models.fno import MyFNO1d

Проведем обучение с разным числом мод.

In [ ]:
modes = [8, 16, 32, 48, 64, 96, 128]

In [ ]:
for modes_num in modes:
    print(f'Training custom FNO with {modes_num} modes')
    print()
    tb_callback = TensorBoardCallback(log_dir=f'./modes_research2/run_{modes_num}_modes')

    my_model = MyFNO1d(n_modes=modes_num, in_channels=3, out_channels=1, n_layers=6, hidden_channels=32, projection_channels=64).to(device)
    my_model = my_model.to(device)

    optimizer = torch.optim.Adam(my_model.parameters(),
                                lr=1e-3)

    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10000, gamma=1)
    # scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=100, eta_min=5e-4)

    h1loss = H1Loss(d=1, reductions='mean')
    l2loss = LpLoss(d=1, p=2, reductions='mean')

    train_loss = h1loss
    eval_losses={'h1': h1loss, 'l2': l2loss}

    trainer = Trainer(model=my_model, n_epochs=202,
                  device=device,
                  wandb_log=False,
                  log_test_interval=3,
                  use_distributed=False,
                  verbose=True,
                  callbacks=[tb_callback])

    trainer.train(train_loader=train_loader, model=my_model, 
                output_encoder=None,
                test_loaders=test_loader,
                optimizer=optimizer,
                scheduler=scheduler,
                regularizer=False,
                training_loss=train_loss,
                eval_losses=eval_losses)

Логи хранятся в директориях modes_research и modes_research2 (два запуска на одном наборе мод). На них можно взглянуть через tensorboard.

In [37]:
!tensorboard --logdir=modes_research

2023-12-02 14:52:04.571522: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-02 14:52:04.990821: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-12-02 14:52:05.539452: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-02 14:52:05.539913: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA n

Графики для одного из запусков:

![](imgs/modes_train.png)

![](imgs/modes_test_l2.png)

![](imgs/modes_test_h1.png)

**Выводы:**

1. Собственная реализация FNO работает корректно и показывает такие же результаты, как FNO из neuralop;
2. Кривая обучения на тесте у запусков с маленьким количеством мод (8, 16, 32) достаточно пологая и в итоге ошибка выше, чем у запусков с большим числом мод;
3. На тесте сильно выделяется запуск с 128 модами (без фильтрации) - он дает наименьшую ошибку, результаты запусков с другим числом мод сопоставимы. 
    
   Однако и здесь прослеживается та же закономерность - на запусках с большим числом мод лучше сходимость.

**Итог:** в данных экспериментах число мод оказывало большое влияние на стабильность обучения и небольшое влияние на перфоманс модели (больше мод - лучше перфоманс и стабильнее обучение).

## Этап 3 - сравнение с 1d-UNet

Для сравнения с FNO я выбрал архитектуру 1d-UNet.

![](imgs/1dunet.png "1D-UNet для размера входа 512")

Создадим модель и обучим её.

In [11]:
from models.unet import UNet1d

In [33]:
st_ch=16
unet = UNet1d(in_channels=3, start_channels=st_ch, out_channels=1)
unet = unet.to(device)

n_params = count_params(unet)
print(f'\nUNet has {n_params} parameters.')
sys.stdout.flush()

optimizer = torch.optim.Adam(unet.parameters(),
                             lr=5e-4)

scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.5)

h1loss = H1Loss(d=1, reductions='mean')
l2loss =LpLoss(d=1, p=2, reductions='mean')

train_loss = h1loss
eval_losses={'h1': h1loss, 'l2': l2loss}

print('\n### MODEL ###\n', unet)
print('\n### OPTIMIZER ###\n', optimizer)
print('\n### SCHEDULER ###\n', scheduler)
print('\n### LOSSES ###')
print(f'\n * Train: {train_loss}')
print(f'\n * Test: {eval_losses}')
sys.stdout.flush()


UNet has 549521 parameters.

### MODEL ###
 UNet1d(
  (encoder): ModuleList(
    (0): EncoderBlock(
      (conv1): Conv1d(3, 16, kernel_size=(3,), stride=(1,), padding=(1,))
      (norm1): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv1d(16, 16, kernel_size=(3,), stride=(1,), padding=(1,))
      (pooling): AvgPool1d(kernel_size=(3,), stride=(2,), padding=(1,))
      (norm2): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act): ReLU()
    )
    (1): EncoderBlock(
      (conv1): Conv1d(16, 32, kernel_size=(3,), stride=(1,), padding=(1,))
      (norm1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv1d(32, 32, kernel_size=(3,), stride=(1,), padding=(1,))
      (pooling): AvgPool1d(kernel_size=(3,), stride=(2,), padding=(1,))
      (norm2): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act): ReLU()
    )
 

In [34]:
unet_exp_num = 3

In [35]:
tb_callback = TensorBoardCallback(log_dir=f'./unet_logs/run{unet_exp_num}_ch{st_ch}')

trainer = Trainer(model=unet, n_epochs=202,
                  device=device,
                  wandb_log=False,
                  log_test_interval=3,
                  use_distributed=False,
                  verbose=True,
                  callbacks=[tb_callback])

trainer.train(train_loader=train_loader, model=unet, 
            output_encoder=None,
            test_loaders=test_loader,
            optimizer=optimizer,
            scheduler=scheduler,
            regularizer=False,
            training_loss=train_loss,
            eval_losses=eval_losses)

using standard method to load data to device.
using standard method to compute loss.
Training on regular inputs (no multi-grid patching).
Training on 400 samples
Testing on [100] samples         on resolutions ['test'].
Training on raw inputs of size x.shape=torch.Size([32, 3, 128]), y.shape=torch.Size([32, 1, 128])
.. patched inputs of size x.shape=torch.Size([32, 3, 128]), y.shape=torch.Size([32, 1, 128])
Raw outputs of size out.shape=torch.Size([32, 1, 128])
.. Processed (unpatched) outputs of size out.shape=torch.Size([32, 1, 128])
[0] time=0.14, avg_loss=0.1464, train_err=0.0739, test_h1=0.0701, test_l2=0.0744
[3] time=0.13, avg_loss=0.0658, train_err=0.0332, test_h1=0.0716, test_l2=0.0679
[6] time=0.13, avg_loss=0.0608, train_err=0.0307, test_h1=0.0636, test_l2=0.0625
[9] time=0.13, avg_loss=0.0569, train_err=0.0287, test_h1=0.0580, test_l2=0.0612
[12] time=0.13, avg_loss=0.0552, train_err=0.0279, test_h1=0.0578, test_l2=0.0620
[15] time=0.13, avg_loss=0.0543, train_err=0.0274, t

In [36]:
!tensorboard --logdir=unet_logs

2023-12-02 14:43:19.679547: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-02 14:43:20.102595: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-12-02 14:43:20.659169: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-02 14:43:20.659688: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA n

Ниже приведены кривые потерь на обучении и тесте для двух запусков с 64 и 32 входными каналами у архитектуры UNet. 

![](imgs/unet_train.png)

![](imgs/unet_test_l2.png)

![](imgs/unet_test_h1.png)

**Выводы:**

1. UNet обучается гораздо нестабильнее, чем FNO.
2. В итоге UNet достигает примерно равного качества с FNO, однако в FNO меньше параметров (200-800K для 32-128 мод у FNO против 2.1M и 8.7M для 32 и 64 входных каналов у UNet), и качество FNO с 128 модами все еще лучше качества всех запусков UNet.

**Итог**: в данной задаче FNO обучается более стабильно, чем UNet, и показывает лучшее качество при меньшем количестве параметров.

![](imgs/fno_mem.png)